<a href="https://colab.research.google.com/github/dustinober1/MNIST_Handwritten_Digits_Udacity/blob/main/MINST_99_TPU.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import tensorflow as tf
from tensorflow.keras.datasets import mnist
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense

In [2]:
# Detect and initialize TPU
try:
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver()  # TPU detection
    print('Running on TPU ', tpu.master())
except ValueError:
    tpu = None

if tpu:
    tf.config.experimental_connect_to_cluster(tpu)
    tf.tpu.experimental.initialize_tpu_system(tpu)
    strategy = tf.distribute.experimental.TPUStrategy(tpu)
else:
    strategy = tf.distribute.get_strategy()  # Default distribution strategy

Running on TPU  


In [3]:
# 1. Load and preprocess the MNIST dataset
(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train = x_train.astype('float32') / 255.0
x_test = x_test.astype('float32') / 255.0
x_train = x_train.reshape(-1, 28, 28, 1)
x_test = x_test.reshape(-1, 28, 28, 1)
y_train = tf.keras.utils.to_categorical(y_train, num_classes=10)
y_test = tf.keras.utils.to_categorical(y_test, num_classes=10)

11490434/11490434 [==============================] - 0s 0us/step


In [4]:
# 2. Define the CNN model
with strategy.scope():  # Create model in TPU's scope
    model = Sequential()
    model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(28, 28, 1)))
    model.add(MaxPooling2D((2, 2)))
    model.add(Conv2D(64, (3, 3), activation='relu'))
    model.add(MaxPooling2D((2, 2)))
    model.add(Flatten())
    model.add(Dense(10, activation='softmax'))

    # 3. Compile the model
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])


In [5]:
# 4. Train the model
model.fit(x_train, y_train, epochs=10, batch_size=32)

Epoch 1/10
1875/1875 [==============================] - 18s 8ms/step - loss: 0.1579 - accuracy: 0.9521
Epoch 2/10
1875/1875 [==============================] - 16s 8ms/step - loss: 0.0576 - accuracy: 0.9823
Epoch 3/10
1875/1875 [==============================] - 16s 8ms/step - loss: 0.0411 - accuracy: 0.9875
Epoch 4/10
1875/1875 [==============================] - 16s 8ms/step - loss: 0.0318 - accuracy: 0.9899
Epoch 5/10
1875/1875 [==============================] - 16s 8ms/step - loss: 0.0255 - accuracy: 0.9918
Epoch 6/10
1875/1875 [==============================] - 15s 8ms/step - loss: 0.0207 - accuracy: 0.9938
Epoch 7/10
1875/1875 [==============================] - 16s 8ms/step - loss: 0.0164 - accuracy: 0.9948
Epoch 8/10
1875/1875 [==============================] - 16s 8ms/step - loss: 0.0133 - accuracy: 0.9960
Epoch 9/10
1875/1875 [==============================] - 16s 8ms/step - loss: 0.0106 - accuracy: 0.9967
Epoch 10/10
1875/1875 [==============================] - 16s 8ms/step - l

In [6]:
# 5. Evaluate the model
loss, accuracy = model.evaluate(x_test, y_test)
print('Test accuracy:', accuracy)

313/313 [==============================] - 6s 15ms/step - loss: 0.0312 - accuracy: 0.9912
Test accuracy: 0.9911999702453613
